In [1]:
# using spacy, compute the number of tokens that don't have an entry in a vocabulary
# (don't have a word embedding) as a measure of noise in the text
# working o test data
import os, pprint, json, pickle
import spacy
path_data = "/data/cm/"
path = "/home/rovera/cm/"

In [5]:
nlp = spacy.load("de_core_news_lg")

In [6]:
def clean_str(string):
    # strip initial/final chars if they are punct
    return string.strip(".,():;—-^׳/\\ *¬„")

stops = set(".,():;—-^׳/\\ *¬„")

In [63]:
doc_count = 0
for file in os.listdir(path_data+"output/test_tools_cm/test_data/"):
    
    if not file.startswith('.'):
        doc_count += 1

In [64]:
# compute oov ratio with spacy word embeddings
overview_sp = {}
glob_oov_sp = {}

for file in os.listdir(path_data+"output/test_tools_cm/test_data/"):
    
    if not file.startswith('.'):
        
        overview_sp[file] = {}
        
        text = open(path_data+"output/test_tools_cm/test_data/"+file).read()
        
        tokenized = []
        voc = {}
        oov_words = {}
        
        doc = nlp(text)
        
        for token in doc:
            
            if not token.is_stop:
            
                word = token.text
                tokenized.append(word)
                
                if word.isupper(): # convert ex. "BUCHDRUCKEREI" into "Buchdruckerei"
                    word = word.lower().capitalize()
                    
                #word = word.strip('.').strip('-') # strip . and - chars
                
                tokenized.append(word)
            
                #try:
                #    word = clean_str(word)
                #except IndexError: # means that the word is an empty string
                #    pass
            
                if nlp.vocab.has_vector(word) is True:
                    voc[word] = True
                else:
                    voc[word] = False
                    if not word in oov_words:
                        oov_words[word] = 0
                    oov_words[word] += 1
                    
                    if not word in glob_oov_sp:
                        glob_oov_sp[word] = 0
                    glob_oov_sp[word] += 1
                    
                
            
            
        overview_sp[file]['len_doc'] = len(tokenized) # tot number of tokens
        
        overview_sp[file]['len_voc'] = len(voc) # vocabulary size
        
        #for word, has_vec in voc.items():
        #    if has_vec is False:
        #        oov += 1
        
        overview_sp[file]['oov_tot_tokens'] = sum(oov_words.values())
        overview_sp[file]['oov_vocabulary'] = len(oov_words)
        overview_sp[file]['oov_tot_ratio'] = sum(oov_words.values())/len(tokenized)
        overview_sp[file]['oov_vocabulary_ratio'] = len(oov_words)/len(voc)

In [41]:
pprint.pprint(overview_sp)

{'10638461_10638480_2728093_055__2744729.txt': {'len_doc': 1594,
                                                'len_voc': 327,
                                                'oov_tot_ratio': 0.06775407779171895,
                                                'oov_tot_tokens': 108,
                                                'oov_vocabulary': 79,
                                                'oov_vocabulary_ratio': 0.2415902140672783},
 '10638461_10638480_2728094_071__2744746.txt': {'len_doc': 2216,
                                                'len_voc': 724,
                                                'oov_tot_ratio': 0.06453068592057762,
                                                'oov_tot_tokens': 143,
                                                'oov_vocabulary': 133,
                                                'oov_vocabulary_ratio': 0.18370165745856354},
 '10638461_6435885_6435898_289__3770808.txt': {'len_doc': 1218,
                                    

                                              'oov_tot_ratio': 0.0784313725490196,
                                              'oov_tot_tokens': 8,
                                              'oov_vocabulary': 8,
                                              'oov_vocabulary_ratio': 0.20512820512820512},
 '9583346_9583348_9581353_110__9581395.txt': {'len_doc': 1792,
                                              'len_voc': 460,
                                              'oov_tot_ratio': 0.47265625,
                                              'oov_tot_tokens': 847,
                                              'oov_vocabulary': 451,
                                              'oov_vocabulary_ratio': 0.9804347826086957},
 '9583346_9583349_9581412_163__9581447.txt': {'len_doc': 1580,
                                              'len_voc': 415,
                                              'oov_tot_ratio': 0.4911392405063291,
                                              'oov_tot

In [2]:
import gensim

In [3]:
we = gensim.models.KeyedVectors.load_word2vec_format(path_data+'word_emb/cc.de.300.vec', binary=False)

In [65]:
# compute oov ratio with FasText word embeddings
overview_fs = {}
glob_oov_fs = {}

for file in os.listdir(path_data+"output/test_tools_cm/test_data/"):
    
    if not file.startswith('.'):
        
        overview_fs[file] = {}
        
        text = open(path_data+"output/test_tools_cm/test_data/"+file).read()
        
        tokenized = []
        voc = {}
        oov_words = {}
        
        doc = nlp(text)
        
        for token in doc:
            
            if not token.is_stop:
                word = token.text
                
                if word.isupper(): # convert ex. "BUCHDRUCKEREI" into "Buchdruckerei"
                    word = word.lower().capitalize()
                    
                #word = word.strip('.').strip('-') # strip . and - chars
                
                tokenized.append(word)
            
                #try:
                #    word = clean_str(word)
                #except IndexError:
                #    pass
            
            
                try:
                    we[word]
                    voc[word] = True
                except KeyError:
                    voc[word] = False
                    if not word in oov_words:
                        oov_words[word] = 0
                    oov_words[word] += 1
                    
                    if not word in glob_oov_fs:
                        glob_oov_fs[word] = 0
                    glob_oov_fs[word] += 1
            
            
        overview_fs[file]['len_doc'] = len(tokenized) # tot number of tokens
        
        overview_fs[file]['len_voc'] = len(voc) # vocabulary size
        
        #for word, has_vec in voc.items():
        #    if has_vec is False:
        #        oov += 1
        
        overview_fs[file]['oov_tot_tokens'] = sum(oov_words.values())
        overview_fs[file]['oov_vocabulary'] = len(oov_words)
        overview_fs[file]['oov_tot_ratio'] = sum(oov_words.values())/len(tokenized)
        overview_fs[file]['oov_vocabulary_ratio'] = len(oov_words)/len(voc)

In [68]:
oov_words = {k: {'c': v, 'state': ''} for k,v in glob_oov_fs.items()}

In [69]:
with open(path_data+"output/nlp_pipeline/oov_words_eval.json", 'w', encoding="utf-8") as outfile:
    json.dump(oov_words, outfile, ensure_ascii=False, indent=2)

In [45]:
pprint.pprint(overview_fs)

{'10638461_10638480_2728093_055__2744729.txt': {'len_doc': 797,
                                                'len_voc': 327,
                                                'oov_tot_ratio': 0.10915934755332497,
                                                'oov_tot_tokens': 87,
                                                'oov_vocabulary': 61,
                                                'oov_vocabulary_ratio': 0.18654434250764526},
 '10638461_10638480_2728094_071__2744746.txt': {'len_doc': 1108,
                                                'len_voc': 724,
                                                'oov_tot_ratio': 0.10108303249097472,
                                                'oov_tot_tokens': 112,
                                                'oov_vocabulary': 102,
                                                'oov_vocabulary_ratio': 0.1408839779005525},
 '10638461_6435885_6435898_289__3770808.txt': {'len_doc': 609,
                                       

In [66]:
# compare the two settings

tot_doc_len = sum([v['len_doc'] for v in overview_sp.values()])
tot_voc_len = sum([v['len_voc'] for v in overview_sp.values()])

avg_voc_ratio_sp = sum([v['oov_vocabulary_ratio'] for v in overview_sp.values()])/doc_count
avg_voc_ratio_fs = sum([v['oov_vocabulary_ratio'] for v in overview_fs.values()])/doc_count
avg_tot_ratio_sp = sum([v['oov_tot_ratio'] for v in overview_sp.values()])/doc_count
avg_tot_ratio_fs = sum([v['oov_tot_ratio'] for v in overview_fs.values()])/doc_count
print("Avg OOV vocabulary ratio Spacy embeddings", str(avg_voc_ratio_sp))
print("Avg OOV vocabulary ratio Fastext embeddings", str(avg_voc_ratio_fs))
print("Avg OOV total word ratio Spacy embeddings", str(avg_tot_ratio_sp))
print("Avg OOV total word ratio Fastext embeddings", str(avg_tot_ratio_fs))

print("OOV vocabulary Spacy", len(glob_oov_sp))
print("OOV vocabulary FasText", len(glob_oov_fs))

print("OOV total number Spacy", sum(glob_oov_sp.values()))
print("OOV total number FasText", sum(glob_oov_fs.values()))

for k in overview_sp.keys():
    
    oov_sp = overview_sp[k]['oov_vocabulary_ratio']
    oov_fs = overview_fs[k]['oov_vocabulary_ratio']
    
    if oov_sp < oov_fs:
        print("Spacy: {}".format(abs(oov_sp-oov_fs)))
    elif oov_fs < oov_sp:
        print("Fastext: {}".format(abs(oov_sp-oov_fs)))
    elif oov_fs == oov_sp:
        print("Same performance.", oov_sp)
    
    

Avg OOV vocabulary ratio Spacy embeddings 0.23871280146726487
Avg OOV vocabulary ratio Fastext embeddings 0.19927538373369455
Avg OOV total word ratio Spacy embeddings 0.08865373910246324
Avg OOV total word ratio Fastext embeddings 0.14762034603785387
OOV vocabulary Spacy 8276
OOV vocabulary FasText 6907
OOV total number Spacy 10563
OOV total number FasText 8740
Fastext: 0.061855670103092786
Same performance. 0.15384615384615385
Fastext: 0.0365296803652968
Fastext: 0.1262135922330097
Fastext: 0.05504587155963303
Fastext: 0.03483043079743353
Fastext: 0.04632152588555857
Fastext: 0.04428044280442803
Fastext: 0.08108108108108109
Fastext: 0.05045871559633028
Fastext: 0.039800995024875635
Fastext: 0.04793028322440088
Fastext: 0.04699453551912569
Fastext: 0.03211009174311927
Fastext: 0.04273504273504272
Fastext: 0.03432494279176203
Fastext: 0.02941176470588236
Fastext: 0.03571428571428571
Fastext: 0.01377410468319562
Fastext: 0.0592334494773519
Fastext: 0.04672897196261683
Fastext: 0.0344827